In [89]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
import os

import selenium

from selenium import webdriver
from selenium.webdriver.common.by import By

import time
from time import sleep
from datetime import datetime

In [2]:
url = 'https://www.getmanfred.com/ofertas-empleo?onlyActive=false&currency=%E2%82%AC'

response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")
response.status_code

200

In [3]:
chrome_driver = "chromedriver.exe"

# Inicializa el navegador
driver = webdriver.Chrome()

# Navega a la página objetivo
driver.get(url)

# Maximizar la página
driver.maximize_window()

In [4]:
sleep (5)

In [9]:
#Aceptar cookies
driver.find_element(by = By.CSS_SELECTOR, value = "#CybotCookiebotDialogBodyButtonAccept").click()

In [10]:
#La página necesita tiempo para actualizarse
sleep(30)

In [13]:
#Obtenemos todas las urls de las ofertas
urls_actualizadas = [f"https://www.getmanfred.com{x.find("a")["href"]}" for x in BeautifulSoup(driver.page_source, "html.parser").find_all("div", class_ = 'react-reveal')]

In [15]:
len (urls_actualizadas)

1294

In [17]:
#Creamos el df con las urls actualizadas
df_urls_actualizadas = pd.DataFrame()
df_urls_actualizadas["url"] = urls_actualizadas

In [19]:
#Leemos las urls que ya tenemos
df_urls_existentes = pd.read_csv('../csv/PARTE_1_1_extraccion_manfred.csv')
urls_existentes = df_urls_existentes['url']

In [21]:
len (urls_existentes)

1279

In [23]:
#Función para obtene las ursl nuevas y el df de todas las urls actualizado
def actualizar_df_urls(df_urls_actualizadas, df_urls_existentes, path_guardar='../csv/urls_MANFRED_actualizado.xlsx'):
    # Detectar urls nuevas
    nuevas_urls = df_urls_actualizadas[~df_urls_actualizadas["url"].isin(df_urls_existentes["url"])]

    # Guardamos el df con las urls 
    df_actualizado = pd.concat([df_urls_existentes, nuevas_urls], ignore_index=True)

    # Guardar si se especifica
    df_actualizado.to_excel(path_guardar, index=False)

    return df_actualizado, nuevas_urls

In [25]:
#Si existen urls nuevas entonces usamos la función

urls_diferencia = set(urls_actualizadas).difference(urls_existentes)

if len(urls_diferencia) > 0:   
    df_actualizado, nuevas_urls = actualizar_df_urls(
        df_urls_actualizadas, 
        df_urls_existentes, 
        path_guardar="../csv/urls_MANFRED_actualizado.xlsx"
    )
else:
    df_actualizado, nuevas_urls = pd.DataFrame(), list()

In [43]:
nuevas_urls

,url
0,https://www.getmanfred.com/ofertas-empleo/7406...
1,https://www.getmanfred.com/ofertas-empleo/7956...
2,https://www.getmanfred.com/ofertas-empleo/7887...
3,https://www.getmanfred.com/ofertas-empleo/7820...
4,https://www.getmanfred.com/ofertas-empleo/7683...
11,https://www.getmanfred.com/ofertas-empleo/7990...
12,https://www.getmanfred.com/ofertas-empleo/7922...
13,https://www.getmanfred.com/ofertas-empleo/7854...
14,https://www.getmanfred.com/ofertas-empleo/7751...
17,https://www.getmanfred.com/ofertas-empleo/7477...


In [33]:
df_urls_actualizadas[~df_urls_actualizadas["url"].isin(df_urls_existentes["url"])]

,url
0,https://www.getmanfred.com/ofertas-empleo/7406...
1,https://www.getmanfred.com/ofertas-empleo/7956...
2,https://www.getmanfred.com/ofertas-empleo/7887...
3,https://www.getmanfred.com/ofertas-empleo/7820...
4,https://www.getmanfred.com/ofertas-empleo/7683...
11,https://www.getmanfred.com/ofertas-empleo/7990...
12,https://www.getmanfred.com/ofertas-empleo/7922...
13,https://www.getmanfred.com/ofertas-empleo/7854...
14,https://www.getmanfred.com/ofertas-empleo/7751...
17,https://www.getmanfred.com/ofertas-empleo/7477...


In [91]:
def scrapear_urls_manfred(df_urls):

    try:
        data = list()
        cont = 1
        for url in df_urls["url"]:
            
            data_manfred = dict()
            print(url)
        
            data_manfred["url"] = url
            soup = BeautifulSoup(requests.get(url).text, "html.parser")
            
            #Nombre
            try:
                data_manfred["nombre"] = soup.find('header', class_='sc-1a753b19-1 edWbWI').find('img') ['alt']
            except:
                data_manfred["nombre"] = np.nan
               
            #Oferta
            try:
                data_manfred["oferta"] = soup.find('h1', class_='sc-f61a956f-0 fsOnFB').text
            except:
                data_manfred["oferta"] = np.nan
            
            #Fecha
            try:
                data_manfred["fecha"] = url.split('-')[-1]
                
            except:
                data_manfred["fecha"] = np.nan
            
            #Fecha de extraccion
            data_manfred["fecha_extraccion"] = datetime.now()
            
            #Salario
            try:
                data_manfred["salario"] = soup.find('a', href='#oferta').text
            except:
                data_manfred["salario"] = np.nan
        
            #Donde
            donde = soup.find_all('a', href='#donde', class_='sc-6d3685c2-2 jPseDG')
            try:
                
                if donde[0].text == '100%':
                    try:
                        data_manfred["remoto"] = True
                        data_manfred["teletrabajo"] = donde[0].text
                        data_manfred["localizacion"] = np.nan
                    except:
                        data_manfred["remoto"] = np.nan
                        data_manfred["teletrabajo"] = np.nan
                else:
                    try:
                        data_manfred["remoto"] = False
                        data_manfred["localizacion"] = donde[0].text
                        data_manfred["teletrabajo"] = donde[1].text if len(donde) > 1 else None
                        
                    except:
                        data_manfred["remoto"] = np.nan
                        data_manfred["localizacion"] = np.nan
                        data_manfred["teletrabajo"] = np.nan
            except:
                data_manfred["remoto"] = np.nan
                data_manfred["localizacion"] = np.nan
                data_manfred["teletrabajo"] = np.nan
                    
            #Día laborable
            try:
                h3 = soup.find('h3', class_='sc-c51cd5df-3 iJStYn', string='Día laborable')
                span = h3.find_next_sibling('span', class_='sc-c51cd5df-4 cPGQFm')
                data_manfred["dia_laborable"] = span.text.strip()
            except:
                data_manfred["dia_laborable"] = np.nan
                
            #Vacaciones
            try:
                h3 = soup.find('h3', class_='sc-c51cd5df-3 iJStYn', string='Vacaciones')
                span = h3.find_next_sibling('span', class_='sc-c51cd5df-4 cPGQFm')
                data_manfred["vacaciones"] = span.text.strip()
            except:
                data_manfred["vacaciones"] = np.nan
        
            
            #Beneficios, idiomas, tecnologias, otras_habilidades
            dicc_tablas = dict()
        
            for section in soup.find_all("section", class_ = "sc-4b1c4ab8-0 hzElAW"):
                dicc_tablas[section.find("h3").text] = list()
            
                if section.find("h3").text == "Beneficios":
                    dicc_tablas[section.find("h3").text].extend([x.text for x in section.find_all("div", class_ = "sc-6083f280-0 gJA-dNI")])
            
                if section.find("h3").text == "Idiomas":
                    dicc_tablas[section.find("h3").text].extend([x.text for x in section.find_all("div", class_ = "sc-24e58b35-4 hjxecG")])
                
                for ul in section.find_all("ul"):
                    dicc_tablas[section.find("h3").text].extend([x.text for x in ul.find_all("h5")])
        
            data_manfred.update(dicc_tablas)
        
            #Textos
            try:
                texto_unido = "\n".join([x.text for x in soup.find_all("section", class_ = "sc-d44be200-0 fqoOyX")])
        
                data_manfred["textos"] = texto_unido
            except:
                data_manfred["textos"] = np.nan
            
            data.append(data_manfred)
           
            sleep(1)
            print(cont)
            cont+=1
            
    except Exception as e:
        print(f"ERROR GENERAL: {e}")
    return data

In [93]:
#Obtenemos los datos de las nuevas ofertas
data_prueba = scrapear_urls_manfred(nuevas_urls)

https://www.getmanfred.com/ofertas-empleo/7406/mavelpoint-lead-frontend-engineer-may25
1
https://www.getmanfred.com/ofertas-empleo/7956/signicat-java-developer-may25
2
https://www.getmanfred.com/ofertas-empleo/7887/eri-bancaire-tech-lead-frontend-abr25
3
https://www.getmanfred.com/ofertas-empleo/7820/bnp-senior-mainframe-specialist-abr25
4
https://www.getmanfred.com/ofertas-empleo/7683/derechocom-sysadmin-abr25
5
https://www.getmanfred.com/ofertas-empleo/7990/abanca-net-developer-may25
6
https://www.getmanfred.com/ofertas-empleo/7922/naiz-fit-ai-engineer-abr25
7
https://www.getmanfred.com/ofertas-empleo/7854/saltoki-data-engineer-abr25
8
https://www.getmanfred.com/ofertas-empleo/7751/driblab-team-lead-computer-visionml-abr25
9
https://www.getmanfred.com/ofertas-empleo/7477/doblemente-net-developer-devops-abr25
10
https://www.getmanfred.com/ofertas-empleo/7170/opositatest-php-backend-developer-may25
11
https://www.getmanfred.com/ofertas-empleo/6887/salsadev-software-developer-abr25
12
h

In [97]:
#Creamos el df con los nuevos datos
df_prueba = pd.json_normalize(data_prueba)
df_prueba

,url,nombre,oferta,fecha,fecha_extraccion,salario,remoto,teletrabajo,localizacion,dia_laborable,vacaciones,Tecnologías,Otras habilidades,Beneficios,textos,Idiomas
0,https://www.getmanfred.com/ofertas-empleo/7406...,MavelPoint,Lead Frontend Engineer,may25,2025-05-09 18:30:05.916211,Hasta €55K,True,100%,NaN,Jornada completa,23 días,"[NextJS, React, PostgreSQL, Azure, nestJS]","[Aprendizaje Continuo, Capacidad de dar feedba...",[Presupuesto para formación],"¿Cómo lo harás?Como decíamos más arriba, nos e...",NaN
1,https://www.getmanfred.com/ofertas-empleo/7956...,Signicat,Java Developer,may25,2025-05-09 18:30:07.398501,€35-45K,True,100%,NaN,Jornada completa,28 días,"[Java, Spring Boot, SQL, RabbitMQ, Google Cloud]","[Aprendizaje Continuo, Trabajo en equipo, Capa...","[Presupuesto para formación, Eventos de formac...","¿Cómo lo harás?\nEso sí, te adelantamos que se...","[Español, Inglés]"
2,https://www.getmanfred.com/ofertas-empleo/7887...,ERI Bancaire,Tech Lead Frontend,abr25,2025-05-09 18:30:08.833586,€55-65K,False,None,Madrid,Jornada completa,23 días,"[JavaScript, CSS, HTML, Vue, React, Angular2+,...","[Adaptabilidad al cambio, Comunicación verbal,...","[Teléfono de empresa, Café, fruta y snacks, Re...",¿Cómo lo harás?En ERI utilizan una aplicación ...,[Inglés]
3,https://www.getmanfred.com/ofertas-empleo/7820...,BNP,Senior Mainframe Specialist,abr25,2025-05-09 18:30:10.314104,Hasta €60K,False,60%,Madrid,Jornada completa,32 días,"[z/OS, DB2, SQL, ServiceNow, Microsoft Active ...","[Trabajo en equipo, Adaptabilidad al cambio, V...","[Seguro de salud, Eventos de formación interno...","¿Cómo lo harás?Aquí no hay ruido, ni burocraci...",[Inglés]
4,https://www.getmanfred.com/ofertas-empleo/7683...,Derecho.com,SysAdmin,abr25,2025-05-09 18:30:14.105330,€45-50K,True,100%,NaN,Jornada completa,23 días,"[Linux, Ansible, Terraform, AWS, Google Cloud,...","[Atención al detalle, Proactividad, Autonomía ...",NaN,¿Cómo lo harás?Trabajarás mano a mano con el C...,NaN
5,https://www.getmanfred.com/ofertas-empleo/7990...,ABANCA,.Net Developer,may25,2025-05-09 18:30:15.670569,€45-54K,True,100%,NaN,Jornada completa,25 días,"[.NET, Azure, Azure CosmosDB, SQL, Git, Docker...","[Capacidad de mentorización, Trabajo en equipo...","[Presupuesto para formación, Seguro de salud, ...",¿Cómo lo harás?Te integrarás en un equipo mult...,NaN
6,https://www.getmanfred.com/ofertas-empleo/7922...,Naiz Fit,AI Engineer,abr25,2025-05-09 18:30:17.151096,€40-45K,True,100%,NaN,Jornada completa,22 días,"[LangChain, Python, ChatGPT, DialogFlow]","[Capacidad de dar feedback, Aprendizaje Contin...","[Presupuesto para formación, Oficina pet-frien...",¿Cómo lo harás?☀️ En el día a día...\nNaiz tie...,[Inglés]
7,https://www.getmanfred.com/ofertas-empleo/7854...,Saltoki,Data Engineer,abr25,2025-05-09 18:30:18.608917,€55-65K,True,100%,NaN,Jornada completa,25 días,"[Snowflake, dbt, SQL, Python, Qlik, AWS, Git]","[Comunicación verbal, Pensamiento analítico, P...","[Teléfono de empresa, Descuento en productos d...","¿Cómo lo harás?Como pasa con Mario y Luigi, aq...",[Inglés]
8,https://www.getmanfred.com/ofertas-empleo/7751...,Driblab,Team Lead Computer Vision/ML,abr25,2025-05-09 18:30:20.067642,Hasta €60K,True,100%,NaN,Jornada completa,25 días,"[OpenCV, Python, SQL, Unix, TensorFlow, PyTorc...","[Gestión de equipo, Capacidad de mentorización...","[Café, fruta y snacks, Eventos de formación in...",¿Cómo lo harás?El equipo de Driblab está compu...,[Inglés]
9,https://www.getmanfred.com/ofertas-empleo/7477...,Doblemente,.Net Developer + DevOps,abr25,2025-05-09 18:30:21.535767,Hasta €35K,True,100%,NaN,Jornada completa,23 días,"[.NET, SQL, Git, API, Azure DevOps, Angular2+]","[Autonomía en el aprendizaje, Atención al deta...","[Presupuesto para formación, Oficina pet-frien...",¿Cómo lo harás?¿Quieres saber más del “intring...,NaN


In [99]:
#Eliminamos los datos que no son necesarios
if 'Guardias' in df_prueba.columns:
    df_prueba = df_prueba.drop('Guardias', axis=1)

In [101]:
if 'Horario' in df_prueba.columns:
    df_prueba = df_prueba.drop('Horario', axis=1)

In [123]:
#Creamos el csv con los datos que utilizaremos para la actualización de la base de datos
df_prueba.to_csv("../csv/PARTE_1_1_extraccion_manfred_actualizacion.csv", index = False)

In [119]:
len (df_prueba)

22

In [121]:
df_prueba

,url,nombre,oferta,fecha,fecha_extraccion,salario,remoto,teletrabajo,localizacion,dia_laborable,vacaciones,Tecnologías,Otras habilidades,Beneficios,textos,Idiomas
0,https://www.getmanfred.com/ofertas-empleo/7406...,MavelPoint,Lead Frontend Engineer,may25,2025-05-09 18:30:05.916211,Hasta €55K,True,100%,NaN,Jornada completa,23 días,"[NextJS, React, PostgreSQL, Azure, nestJS]","[Aprendizaje Continuo, Capacidad de dar feedba...",[Presupuesto para formación],"¿Cómo lo harás?Como decíamos más arriba, nos e...",NaN
1,https://www.getmanfred.com/ofertas-empleo/7956...,Signicat,Java Developer,may25,2025-05-09 18:30:07.398501,€35-45K,True,100%,NaN,Jornada completa,28 días,"[Java, Spring Boot, SQL, RabbitMQ, Google Cloud]","[Aprendizaje Continuo, Trabajo en equipo, Capa...","[Presupuesto para formación, Eventos de formac...","¿Cómo lo harás?\nEso sí, te adelantamos que se...","[Español, Inglés]"
2,https://www.getmanfred.com/ofertas-empleo/7887...,ERI Bancaire,Tech Lead Frontend,abr25,2025-05-09 18:30:08.833586,€55-65K,False,None,Madrid,Jornada completa,23 días,"[JavaScript, CSS, HTML, Vue, React, Angular2+,...","[Adaptabilidad al cambio, Comunicación verbal,...","[Teléfono de empresa, Café, fruta y snacks, Re...",¿Cómo lo harás?En ERI utilizan una aplicación ...,[Inglés]
3,https://www.getmanfred.com/ofertas-empleo/7820...,BNP,Senior Mainframe Specialist,abr25,2025-05-09 18:30:10.314104,Hasta €60K,False,60%,Madrid,Jornada completa,32 días,"[z/OS, DB2, SQL, ServiceNow, Microsoft Active ...","[Trabajo en equipo, Adaptabilidad al cambio, V...","[Seguro de salud, Eventos de formación interno...","¿Cómo lo harás?Aquí no hay ruido, ni burocraci...",[Inglés]
4,https://www.getmanfred.com/ofertas-empleo/7683...,Derecho.com,SysAdmin,abr25,2025-05-09 18:30:14.105330,€45-50K,True,100%,NaN,Jornada completa,23 días,"[Linux, Ansible, Terraform, AWS, Google Cloud,...","[Atención al detalle, Proactividad, Autonomía ...",NaN,¿Cómo lo harás?Trabajarás mano a mano con el C...,NaN
5,https://www.getmanfred.com/ofertas-empleo/7990...,ABANCA,.Net Developer,may25,2025-05-09 18:30:15.670569,€45-54K,True,100%,NaN,Jornada completa,25 días,"[.NET, Azure, Azure CosmosDB, SQL, Git, Docker...","[Capacidad de mentorización, Trabajo en equipo...","[Presupuesto para formación, Seguro de salud, ...",¿Cómo lo harás?Te integrarás en un equipo mult...,NaN
6,https://www.getmanfred.com/ofertas-empleo/7922...,Naiz Fit,AI Engineer,abr25,2025-05-09 18:30:17.151096,€40-45K,True,100%,NaN,Jornada completa,22 días,"[LangChain, Python, ChatGPT, DialogFlow]","[Capacidad de dar feedback, Aprendizaje Contin...","[Presupuesto para formación, Oficina pet-frien...",¿Cómo lo harás?☀️ En el día a día...\nNaiz tie...,[Inglés]
7,https://www.getmanfred.com/ofertas-empleo/7854...,Saltoki,Data Engineer,abr25,2025-05-09 18:30:18.608917,€55-65K,True,100%,NaN,Jornada completa,25 días,"[Snowflake, dbt, SQL, Python, Qlik, AWS, Git]","[Comunicación verbal, Pensamiento analítico, P...","[Teléfono de empresa, Descuento en productos d...","¿Cómo lo harás?Como pasa con Mario y Luigi, aq...",[Inglés]
8,https://www.getmanfred.com/ofertas-empleo/7751...,Driblab,Team Lead Computer Vision/ML,abr25,2025-05-09 18:30:20.067642,Hasta €60K,True,100%,NaN,Jornada completa,25 días,"[OpenCV, Python, SQL, Unix, TensorFlow, PyTorc...","[Gestión de equipo, Capacidad de mentorización...","[Café, fruta y snacks, Eventos de formación in...",¿Cómo lo harás?El equipo de Driblab está compu...,[Inglés]
9,https://www.getmanfred.com/ofertas-empleo/7477...,Doblemente,.Net Developer + DevOps,abr25,2025-05-09 18:30:21.535767,Hasta €35K,True,100%,NaN,Jornada completa,23 días,"[.NET, SQL, Git, API, Azure DevOps, Angular2+]","[Autonomía en el aprendizaje, Atención al deta...","[Presupuesto para formación, Oficina pet-frien...",¿Cómo lo harás?¿Quieres saber más del “intring...,NaN


In [109]:
#Creamos el df de todos los datos y actualizamos el csv

df_urls_existentes = pd.read_csv('../csv/PARTE_1_1_extraccion_manfred.csv')

df_todo = pd.concat([df_urls_existentes, df_prueba], ignore_index = True).drop_duplicates("url")

df_todo.to_csv("../csv/PARTE_1_1_extraccion_manfred.csv", index = False)


In [111]:
len (df_todo)

1301